In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('training_images/training_images.csv')

In [29]:
df
X = df.drop('label', axis=1)
y = pd.get_dummies(df['label'])

In [30]:
model=Sequential()
model.add(Dense(64, input_dim=1024, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(36, activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=20)

Epoch 1/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.2394 - accuracy: 0.9260 - val_loss: 0.4338 - val_accuracy: 0.8835
Epoch 2/20
1800/1800 [==============================] - 3s 2ms/step - loss: 0.2235 - accuracy: 0.9302 - val_loss: 0.4323 - val_accuracy: 0.8847
Epoch 3/20
1800/1800 [==============================] - 3s 2ms/step - loss: 0.2132 - accuracy: 0.9336 - val_loss: 0.4261 - val_accuracy: 0.8874
Epoch 4/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.2018 - accuracy: 0.9364 - val_loss: 0.4609 - val_accuracy: 0.8785
Epoch 5/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1887 - accuracy: 0.9411 - val_loss: 0.4284 - val_accuracy: 0.8870
Epoch 6/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1802 - accuracy: 0.9427 - val_loss: 0.4559 - val_accuracy: 0.8824
Epoch 7/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1732 - accuracy: 0.9452 - val_loss: 0.4727 - val_accuracy:

In [35]:
import pickle

# Serialize the model to a pkl file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://85c51ba4-72a5-4c86-a45e-a94ee1110785/assets


In [44]:
model.predict(X_test)

450/450 [==============================] - 1s 2ms/step


array([[8.92218395e-22, 3.60060901e-17, 1.05344045e-16, ...,
        5.38614986e-12, 8.55593540e-21, 3.13213409e-12],
       [2.05810008e-10, 1.54949839e-22, 1.71338185e-19, ...,
        7.41313749e-15, 4.51903823e-18, 4.76231286e-08],
       [1.28653778e-31, 6.56112052e-18, 3.78076082e-20, ...,
        5.70357576e-18, 4.13482873e-25, 6.03169519e-26],
       ...,
       [4.84435176e-08, 3.52024631e-06, 4.85917251e-10, ...,
        1.03906968e-04, 5.41429372e-08, 9.64086950e-01],
       [1.72968096e-27, 7.37499395e-10, 8.81625883e-11, ...,
        1.21466976e-32, 2.86553279e-13, 7.20565735e-27],
       [1.46324307e-22, 1.04945962e-10, 1.06954878e-09, ...,
        1.68638712e-15, 2.67742681e-17, 8.91493905e-19]], dtype=float32)

In [45]:
!pip install PIL


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [46]:
from PIL import Image
img=Image.open('ka.png').convert('L')

In [47]:
img_array=np.array(img).flatten()/255

In [48]:
img_array.shape

(1024,)

In [49]:
model.predict(img_array.reshape(1,1024))

1/1 [==============================] - 0s 30ms/step


array([[2.5822997e-02, 1.4709172e-02, 2.4908120e-05, 6.4417117e-11,
        4.2512247e-04, 1.4199319e-04, 2.0338500e-06, 3.4243116e-05,
        7.0410693e-01, 1.4415123e-03, 5.9563854e-10, 8.7991064e-10,
        2.4485640e-05, 1.1902259e-13, 5.7088374e-07, 2.3469776e-10,
        4.8778093e-09, 2.3699126e-06, 3.2649663e-08, 2.3177531e-06,
        8.6086330e-13, 2.2985277e-01, 2.9081395e-06, 5.4153102e-04,
        1.5271539e-04, 6.3762620e-08, 2.0086417e-09, 7.0791871e-06,
        3.5103873e-09, 2.1311427e-02, 1.8996818e-08, 2.1948169e-04,
        1.7575806e-04, 3.8583067e-04, 3.1122117e-04, 3.0057208e-04]],
      dtype=float32)

In [53]:
model.save("model.h5")

In [55]:
from tensorflow.keras.models import load_model

# Load the saved model
model1 = load_model("model.h5")

# Check the model summary
model1.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                65600     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 36)                2340      
                                                                 
Total params: 72,100
Trainable params: 72,100
Non-trainable params: 0
_________________________________________________________________


In [56]:
model1.predict(img_array.reshape(1,1024))

1/1 [==============================] - 0s 159ms/step


array([[2.5822997e-02, 1.4709172e-02, 2.4908120e-05, 6.4417117e-11,
        4.2512247e-04, 1.4199319e-04, 2.0338500e-06, 3.4243116e-05,
        7.0410693e-01, 1.4415123e-03, 5.9563854e-10, 8.7991064e-10,
        2.4485640e-05, 1.1902259e-13, 5.7088374e-07, 2.3469776e-10,
        4.8778093e-09, 2.3699126e-06, 3.2649663e-08, 2.3177531e-06,
        8.6086330e-13, 2.2985277e-01, 2.9081395e-06, 5.4153102e-04,
        1.5271539e-04, 6.3762620e-08, 2.0086417e-09, 7.0791871e-06,
        3.5103873e-09, 2.1311427e-02, 1.8996818e-08, 2.1948169e-04,
        1.7575806e-04, 3.8583067e-04, 3.1122117e-04, 3.0057208e-04]],
      dtype=float32)